# Konjunkturprognose Tracker
November 2025

## 0) Setup


In [185]:
import pandas as pd
from playwright.async_api import async_playwright
import time
import aiohttp
import asyncio
import requests
import io
from io import StringIO
import numpy as np
import gspread
from gspread_dataframe import set_with_dataframe
from google.oauth2.service_account import Credentials
import os
import json 
import pygsheets
from datawrapper import Datawrapper

## 1) Datengenerierung

Zuerst lade ich die Konjunkturprognosen aus dem Google Sheet. 

In [186]:
# Daten von Google Sheets laden
SHEET_ID = "1_GuhaEY2LHJeSgae8tiioBCTsnQ3YzrC3BbVCybY4A0" 
gc = pygsheets.authorize(service_file='/Users/bb/Desktop/handelsblatt/Konjunkturprognosetracker/credentials.json')
sh = gc.open_by_key(SHEET_ID)
ws1 = sh[3] 
df_prog = pd.DataFrame(ws1.get_all_records())

# Spalten auswählen und und numerische Umwandlung
df_prog = df_prog[["Year", "Value", "Institute", "Release Season", "Horizon", "Vintage"]]
df_prog["Year"] = pd.to_numeric(df_prog["Year"], errors="coerce")
df_prog['Vintage'] = pd.to_datetime(df_prog['Vintage'], errors='coerce')

# Datensatz sortieren
df_prog = df_prog.sort_values(['Year', 'Institute', 'Vintage'], ascending=[False, True, False])

df_prog

,Year,Value,Institute,Release Season,Horizon,Vintage
892,2027,1.6,DIW,Winter,2,2025-12-12
891,2027,1.8,DIW,Autumn,2,2025-09-04
879,2027,1.0,IWH,Winter,2,2025-12-11
878,2027,0.6,IWH,Autumn,2,2025-09-04
925,2027,1.3,Kiel Institute,Winter,2,2025-12-11
...,...,...,...,...,...,...
821,2000,3.2,RWI,Winter,0,1999-02-01
161,2000,3.0,ifo,Winter,0,2000-12-20
346,2000,2.7,ifo,Summer,0,2000-07-27
863,2000,2.7,ifo,Winter,1,1999-12-21


Dann lade ich separat die wahren Konjunkturdaten.

In [187]:
# Daten laden
data = sh[4] 
df_real = pd.DataFrame(data).iloc[27:45, [0, 7]]

# Spalten umbennen, so dass sie df_prog entsprechen
df_real = df_real.rename(columns={df_real.columns[0]: 'Year', df_real.columns[1]: 'Value'})

# Daten säubern und numerisch umwandeln
df_real['Year'] = pd.to_numeric(df_real['Year'], errors='coerce')
df_real['Value'] = df_real['Value'].str.replace(',', '.', regex=False)
df_real['Value'] = df_real['Value'].str.replace('– ', '-', regex=False)  
df_real['Value'] = pd.to_numeric(df_real['Value'], errors='coerce')
df_real


,Year,Value
27,2005,0.7
28,2006,3.8
29,2007,3.0
30,2008,1.0
31,2009,-5.7
32,2010,4.2
33,2011,3.9
34,2012,0.4
35,2013,0.4
36,2014,2.2


## 2) Datenumwandlung

### 2.1) Grafiken 1: Mögliche Abweichung der neuesten Prognosen

Zuerst berechne den Median der Differenz der Prognose vom eingetroffenen Wert. Ich unterscheide hierfür nach Institut, Vorhersageentfernung und Abweichungsrichtung.

In [188]:
# Daten zusammenführen
df = df_prog.merge(df_real[['Year', 'Value']], on='Year', suffixes=('_prog', '_real'))

# Abweichung berechnen
df['diff'] = df['Value_prog'] - df['Value_real']

# in positive und negative Abweichungen unterteilen
df['higher'] = (df['diff'] > 0).astype(int)

# Median Abweichung pro Institut, Prognosezeitraum und positive/negative Abweichung
df['median_diff'] = df.groupby(['Institute', 'Horizon', 'higher'])['diff'].transform('median')

# Nur zur Überprüfung: Berechnung der Anzahl der Beobachtungen pro Median-Berechnung; Missing setzen, wenn weniger als 5 Beobachtungen vorhanden sind
df['count'] = df.groupby(['Institute', 'Horizon', 'higher'])['diff'].transform('count')
df['flag_low_count'] = df['count'] < 5
df.loc[df['count'] < 5, 'diff'] = np.nan

Dann erstelle ich ein Datensatz, der nur noch diese Medianabweichungen enthält. 

In [189]:
# nur die relevanten Spalten zur Einordnung der Medianabweichung behalten
df_medians = df.loc[:, ["Institute", "Horizon", "higher", "median_diff"]]

# die daraus entstehenden Duplikate entfernen, damit jeweils jeder Code nur einmal vorkommt
df_medians = df_medians.drop_duplicates()

# Reihen zusammenfassen und separate Spalten für positive und negative Medianabweichungen erstellen
df_medians = df_medians.pivot(index=["Institute", "Horizon"],columns="higher", values="median_diff")
df_medians = df_medians.rename(columns={0: "median_diff_lower", 1: "median_diff_higher"})

df_medians = df_medians.reset_index() 

Nun spielen wir die Medianabweichungen in den Datensatz aller Konjunkturprognosen und errechnen anhand dieser die hypothetischen Wachstumswerte, wenn die typische obere und untere Abweichung eintreten würde.

In [190]:
# Datensatz absichern
df_prog_graph1 = df_prog.copy()

# Die Medianabweichungen an den Datensatz anhängen
df_graph1 = df_prog_graph1.merge(df_medians, on=["Institute", "Horizon"], how="left")

# Die typischen Abweichungsbereiche berechnen
df_graph1["Lower"] = df_graph1["Value"] + df_graph1["median_diff_lower"]
df_graph1["Higher"] = df_graph1["Value"] + df_graph1["median_diff_higher"]

# Nur die relevanten Spalten in sinnvoller Reihenfolge behalten
df_graph1 = df_graph1[["Year", "Institute", "Lower", "Value", "Higher", "Horizon"]]

df_graph1

,Year,Institute,Lower,Value,Higher,Horizon
0,2027,DIW,0.10,1.6,2.40,2
1,2027,DIW,0.30,1.8,2.60,2
2,2027,IWH,-0.80,1.0,2.10,2
3,2027,IWH,-1.20,0.6,1.70,2
4,2027,Kiel Institute,-0.45,1.3,2.40,2
...,...,...,...,...,...,...
990,2000,RWI,2.50,3.2,3.75,0
991,2000,ifo,2.40,3.0,3.50,0
992,2000,ifo,2.10,2.7,3.20,0
993,2000,ifo,1.60,2.7,4.20,1


Nun sortiere ich die Prognosen nach Jahren in Datensätze.   
Ich benötige nur die Werte für Jahre, in denen noch kein wahrer Wert bekannt ist.  Diese Jahre setze ich als Global, da sie nicht konstant sind, sondern sich mit der Zeit verändern Da Konjunkturprognosen immer zwei Jahre im Voraus abgegeben, sind das die drei höchsten Jahre im Datensatz.

In [191]:
years = sorted(df_graph1["Year"].unique(), reverse=True)
year2, year1, year0 = years[:3]

#Zur Überprüfung
print("Wir haben das Jahr " + str(year0) + " und unsere Prognosejahre sind " + str(year0) + ", " + str(year1) + " und " + str(year2) + ".")

Wir haben das Jahr 2025 und unsere Prognosejahre sind 2025, 2026 und 2027.


Konjunkturprognosen werden viermal im Jahr abgegeben. Wir wollen uns aber nur eine, die aktuellste, anschauen. 

In [192]:
df_graph1 = df_graph1.groupby(['Year', 'Institute', "Horizon"], as_index=False).first()

Nun erstellen wir mit diesen Prognosen einzelne Datensets pro Jahr. 

In [193]:
# Datenset einzelnd pro Jahr trennen; Werte runden
df_year0 = df_graph1[df_graph1['Year'] == year0][df_graph1['Horizon'] == 0].round(2).copy()
df_year1 = df_graph1[df_graph1['Year'] == year1][df_graph1['Horizon'] == 1].round(2).copy()
df_year2 = df_graph1[df_graph1['Year'] == year2][df_graph1['Horizon'] == 2].round(2).copy()  

/var/folders/01/8pcv5mtj69j_y6v97hckzr1r0000gn/T/ipykernel_14570/1941179115.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_year0 = df_graph1[df_graph1['Year'] == year0][df_graph1['Horizon'] == 0].round(2).copy()
/var/folders/01/8pcv5mtj69j_y6v97hckzr1r0000gn/T/ipykernel_14570/1941179115.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_year1 = df_graph1[df_graph1['Year'] == year1][df_graph1['Horizon'] == 1].round(2).copy()
/var/folders/01/8pcv5mtj69j_y6v97hckzr1r0000gn/T/ipykernel_14570/1941179115.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_year2 = df_graph1[df_graph1['Year'] == year2][df_graph1['Horizon'] == 2].round(2).copy()


Zum Schluss errechne ich für jedes Datenset noch einen Durchschnitt und hänge ihn an. 

In [194]:
# Loop zur Berechnung der Medianwerte
for name in ["df_year0", "df_year1", "df_year2"]:
    df_tmp = locals()[name]

    # Horizon Spalte entfernen, da irrelevant
    df_tmp = df_tmp[["Year", "Institute", "Lower", "Value", "Higher"]]

    # Mediane berechnen
    avg_row = (df_tmp.groupby("Year", as_index=False)[["Lower", "Value", "Higher"]].median())
    avg_row["Institute"] = "Median"

    # Extra Spalte hinzufügen, der später den Median kennzeichnet
    avg_row["Type"] = "Median"

    # Mediane hinzufügen
    avg_row = avg_row[df_tmp.columns]
    
    locals()[name] = pd.concat([df_tmp, avg_row], ignore_index=True)

    


Zur Überprüfung:

In [195]:
df_year0

,Year,Institute,Lower,Value,Higher
0,2025,DIW,-0.5,0.2,1.00
1,2025,IWH,-0.5,0.2,0.70
2,2025,Kiel Institute,-0.5,0.1,0.55
3,2025,RWI,-0.6,0.1,0.65
4,2025,ifo,-0.5,0.1,0.60
5,2025,Median,-0.5,0.1,0.65


In [196]:
df_year1

,Year,Institute,Lower,Value,Higher
0,2026,DIW,0.10,1.3,2.6
1,2026,IWH,-0.45,1.0,2.1
2,2026,Kiel Institute,-0.00,1.0,2.2
3,2026,RWI,-0.10,1.0,2.3
4,2026,ifo,-0.30,0.8,2.3
5,2026,Median,-0.10,1.0,2.3


In [197]:
df_year2

,Year,Institute,Lower,Value,Higher
0,2027,DIW,0.10,1.6,2.40
1,2027,IWH,-0.80,1.0,2.10
2,2027,Kiel Institute,-0.45,1.3,2.40
3,2027,RWI,-0.30,1.4,2.25
4,2027,ifo,-0.65,1.1,2.40
5,2027,Median,-0.45,1.3,2.40


### 2.2) Grafik2: Prognosewerte über die Zeit: Berechnung der jährlichen Medianprognose

Zuerst berechnen wir für jedes Jahr den Median der Werte aller Konjunkturprognosen, die für ein Folgejahr getroffen wurden. 

In [198]:
df_prog_graph2 = df_prog.copy()

# Nur die Vorjahresprognosen behalten
df_prog_graph2 = df_prog_graph2[df_prog_graph2['Horizon'] == 1]

# Medianprognosewerte 
df_prog_graph2['median_prog'] = df_prog_graph2.groupby(['Year'])["Value"].transform('median')

# Nur die Medianprognosen pro Jahr behalten
df_prog_graph2 = (df_prog_graph2.loc[:, ["Year", "median_prog"]].drop_duplicates())

df_prog_graph2


,Year,median_prog
485,2026,1.25
481,2025,1.05
477,2024,1.40
447,2023,1.15
632,2022,4.00
658,2021,4.15
657,2020,1.40
656,2019,1.70
622,2018,2.00
654,2017,1.50


Jetzt holen wir die eingetroffenen Werte hinzu. 

In [207]:
# Die Medianabweichungen an den Datensatz anhängen
df_graph2 = df_prog_graph2.merge(df_real, on=["Year"], how="right")

df_graph2 = df_graph2[df_graph2["Year"] > 2014]
df_graph2

,Year,median_prog,Value
10,2015,1.95,1.5
11,2016,1.90,2.2
12,2017,1.50,2.7
13,2018,2.00,1.0
14,2019,1.70,1.1
15,2020,1.40,-3.8
16,2021,4.15,3.2
17,2022,4.00,1.8


## 3) Visualize

In [208]:
wkYEAR0 = sh[0] 
wkYEAR1 = sh[1] 
wkYEAR2 = sh[2] 
wkALL = sh[5]
wkYEAR0.clear()
wkYEAR1.clear()
wkYEAR2.clear()
wkALL.clear()

wkYEAR0.set_dataframe(df_year0, (1,1)) 
wkYEAR1.set_dataframe(df_year1, (1,1))
wkYEAR2.set_dataframe(df_year2, (1,1))
wkALL.set_dataframe(df_graph2, (1,1))

In [209]:
from datawrapper import Datawrapper

CREDENTIALS_PATH = "/Users/bb/Desktop/handelsblatt/Konjunkturprognosetracker/credentials_dw.json"

with open(CREDENTIALS_PATH, "r") as f:
    data = json.load(f)
    DWKEY = data["key"]


Aktuelles Jahr

In [210]:

dw = Datawrapper(access_token=DWKEY)
allcharts=dw.get_charts(limit=500)
filtered_charts=list(filter(lambda item: item["folderId"] == "onkX1", allcharts["list"]))
for item in (filtered_charts):
  print(item["id"])
  dw.publish_chart(chart_id=item["id"])

Aktuelles Jahr + 1

In [211]:

dw = Datawrapper(access_token=DWKEY)
allcharts=dw.get_charts(limit=500)
filtered_charts=list(filter(lambda item: item["folderId"] == "s4Vk3", allcharts["list"]))
for item in (filtered_charts):
  print(item["id"])
  dw.publish_chart(chart_id=item["id"])

Aktuelles Jahr + 2

In [212]:

dw = Datawrapper(access_token=DWKEY)
allcharts=dw.get_charts(limit=500)
filtered_charts=list(filter(lambda item: item["folderId"] == "3Yzvz", allcharts["list"]))
for item in (filtered_charts):
  print(item["id"])
  dw.publish_chart(chart_id=item["id"])

Alle Jahre

In [213]:
dw = Datawrapper(access_token=DWKEY)
allcharts=dw.get_charts(limit=500)
filtered_charts=list(filter(lambda item: item["folderId"] == "3Yzvz", allcharts["list"]))
for item in (filtered_charts):
  print(item["id"])
  dw.publish_chart(chart_id=item["id"])